In [1]:
import torch
import visdom
from torch.utils.data import DataLoader
from torch import nn,optim
from torchvision import transforms, datasets

/home/halcyon/ProgramFiles/anaconda3/envs/ngsim/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:

mnist_train=datasets.MNIST('mnist',True,transform=transforms.Compose([transforms.ToTensor()]),download=True)
mnist_train=DataLoader(mnist_train,batch_size=32,shuffle=True)

mnist_test=datasets.MNIST('mnist',False,transform=transforms.Compose([transforms.ToTensor()]),download=True)
mnist_test=DataLoader(mnist_test,batch_size=32,shuffle=True)

# x,_=iter(mnist_train).next()
# print('x',x.shape)

x=next(iter(mnist_train)) #list
# type(x)
x=x[0]

In [13]:
class vae(nn.Module):
    def __init__(self):
        super(vae,self).__init__()
        self.encoder=nn.Sequential(
            nn.Linear(784, 256),
            nn.ReLU(),
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Linear(64, 20),
            nn.ReLU()
        )

        self.decoder=nn.Sequential(
            nn.Linear(10, 64),
            nn.ReLU(),
            nn.Linear(64, 256),
            nn.ReLU(),
            nn.Linear(256, 784),
            nn.Sigmoid()
        )
        self.criteon = nn.MSELoss()

    def forward(self,x):
        batchsize=x.size(0)
        x=x.view(batchsize,784)  #[b, 1, 28, 28]->[b, 784]
        h_ = self.encoder(x) #[b,784]->[b,20]
        mu, sigma = h_.chunk(2, dim=1) # .chunk把1维拆两份   # [b, 20] => [b, 10] and [b, 10]
        h=mu+sigma*torch.randn_like(sigma)

        x_hat = self.decoder(h) # [b, 10] => [b, 784]
        x_hat=x_hat.view(batchsize,1,28,28) # [b, 784] => [b,1,28,28]

        kld = 0.5 * torch.sum(
            torch.pow(mu, 2) +
            torch.pow(sigma, 2) -
            torch.log(1e-8 + torch.pow(sigma, 2)) - 1
        ) / (batchsize*28*28)

        return x_hat, kld



In [14]:
device=torch.device('cpu')
model=vae().to(device)
criterion=nn.MSELoss()
optimizer=optim.Adam(model.parameters(),lr=1e-3)
print(model)

vae(
  (encoder): Sequential(
    (0): Linear(in_features=784, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=20, bias=True)
    (5): ReLU()
  )
  (decoder): Sequential(
    (0): Linear(in_features=10, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=784, bias=True)
    (5): Sigmoid()
  )
  (criteon): MSELoss()
)


In [15]:
viz=visdom.Visdom()

Setting up a new session...
Traceback (most recent call last):
  File "/home/halcyon/ProgramFiles/anaconda3/envs/highwayenv/lib/python3.9/site-packages/urllib3/connection.py", line 174, in _new_conn
    conn = connection.create_connection(
  File "/home/halcyon/ProgramFiles/anaconda3/envs/highwayenv/lib/python3.9/site-packages/urllib3/util/connection.py", line 95, in create_connection
    raise err
  File "/home/halcyon/ProgramFiles/anaconda3/envs/highwayenv/lib/python3.9/site-packages/urllib3/util/connection.py", line 85, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/halcyon/ProgramFiles/anaconda3/envs/highwayenv/lib/python3.9/site-packages/urllib3/connectionpool.py", line 703, in urlopen
    httplib_response = self._make_request(
  File "/home/halcyon/ProgramFiles/anaconda3/envs/highwayenv/lib/python3.9/site-packages

Exception in user code:
------------------------------------------------------------


In [17]:
for epoch in range(100):
    for batchidx,(x,_) in enumerate(mnist_train):
        x = x.to(device)
        x_hat, kld = model(x)
        loss = criterion(x_hat, x)

        if kld is not None:
                elbo = - loss - 1.0 * kld
                loss = - elbo

        # backprop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(epoch, 'loss:', loss.item(), 'kld:', kld.item())

    # x, _ = iter(mnist_test).next()
    x=next(iter(mnist_test))[0]
    x = x.to(device)
    with torch.no_grad():
        x_hat, kld = model(x)
    viz.images(x, nrow=8, win='x', opts=dict(title='x'))
    viz.images(x_hat, nrow=8, win='x_hat', opts=dict(title='x_hat'))

    #28min



0 loss: 0.047037139534950256 kld: 0.0070672230795025826
Exception in user code:
------------------------------------------------------------
Exception in user code:
------------------------------------------------------------


Traceback (most recent call last):
  File "/home/halcyon/ProgramFiles/anaconda3/envs/highwayenv/lib/python3.9/site-packages/urllib3/connection.py", line 174, in _new_conn
    conn = connection.create_connection(
  File "/home/halcyon/ProgramFiles/anaconda3/envs/highwayenv/lib/python3.9/site-packages/urllib3/util/connection.py", line 95, in create_connection
    raise err
  File "/home/halcyon/ProgramFiles/anaconda3/envs/highwayenv/lib/python3.9/site-packages/urllib3/util/connection.py", line 85, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/halcyon/ProgramFiles/anaconda3/envs/highwayenv/lib/python3.9/site-packages/urllib3/connectionpool.py", line 703, in urlopen
    httplib_response = self._make_request(
  File "/home/halcyon/ProgramFiles/anaconda3/envs/highwayenv/lib/python3.9/site-packages/urllib3/connectionpool.py",

1 loss: 0.046520382165908813 kld: 0.008815333247184753
2 loss: 0.04873841255903244 kld: 0.009637745097279549
3 loss: 0.047776322811841965 kld: 0.0103320786729455
4 loss: 0.04716676101088524 kld: 0.00994142796844244
5 loss: 0.04519200697541237 kld: 0.009844575077295303
6 loss: 0.04340957850217819 kld: 0.010048337280750275
7 loss: 0.052362650632858276 kld: 0.010751135647296906
8 loss: 0.047785237431526184 kld: 0.009688310325145721
9 loss: 0.047456495463848114 kld: 0.010068468749523163
10 loss: 0.04745426028966904 kld: 0.010793942958116531
11 loss: 0.043814003467559814 kld: 0.010145558044314384
12 loss: 0.0428200326859951 kld: 0.010678955353796482
13 loss: 0.05002981796860695 kld: 0.010868639685213566
14 loss: 0.04393035173416138 kld: 0.010818222537636757
15 loss: 0.0401725098490715 kld: 0.010298332199454308
16 loss: 0.04468574374914169 kld: 0.010775149799883366
17 loss: 0.04109886288642883 kld: 0.010970834642648697
18 loss: 0.0408400222659111 kld: 0.010807771235704422
19 loss: 0.04954159

In [ ]:
# 在终端里python -m visdom.server